In [1]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
#!unzip '/content/drive/My Drive/all data/clean_bert'
import pickle

with open('br_tr' , 'rb') as f:
  tr = pickle.load(f)
len(tr)

25000

In [4]:
import nltk

from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# TOKEN LEVEL FEATURES

In [0]:
#!pip install bert-embedding
#!pip install mxnet-cu100
#!pip uninstall mxnet

In [0]:
import mxnet as mx

from bert_embedding import BertEmbedding
bert_embedding = BertEmbedding()
ctx = mx.gpu(0)
bert = BertEmbedding(ctx=ctx)

/usr/local/lib/python3.6/dist-packages/mxnet/gluon/utils.py:334: UserWarning: File /root/.mxnet/models/bert_12_768_12_book_corpus_wiki_en_uncased-75cc780f.zip exists in file system so the downloaded file is deleted
  'File {} exists in file system so the downloaded file is deleted'.format(fname))


In [0]:
import numpy as np
bert_embedding = BertEmbedding()
t = tr[5000:10000]
k = 0

doc_vecs = []
for doc in t:
  sentences = sent_tokenize(doc)
  len(sentences)
  result = bert_embedding(sentences)
  t = np.zeros(768)
  l = 0
  for i in range(len(result)):
    l += len(result[i][1])
    t += np.sum(result[i][1],axis = 0)
  doc_vecs.append(t/l)
  k+=1
  if k%100 == 0:print(k)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000


In [0]:
with open('/content/drive/My Drive/all data/2token' , 'wb') as f:
  pickle.dump(doc_vecs,f)

# SENTECE LEVEL FEATURES

In [0]:
#!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [11]:
import numpy as np
k = 0
doc_vecs = []
for doc in te:
  sentences = sent_tokenize(doc)
  sentence_embeddings = model.encode(sentences)
  doc_vecs.append(np.mean(sentence_embeddings,axis = 0))
  k+=1
  if k%200 == 0:print(k)


200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400
12600
12800
13000
13200
13400
13600
13800
14000
14200
14400
14600
14800
15000
15200
15400
15600
15800
16000
16200
16400
16600
16800
17000
17200
17400
17600
17800
18000
18200
18400
18600
18800
19000
19200
19400
19600
19800
20000
20200
20400
20600
20800
21000
21200
21400
21600
21800
22000
22200
22400
22600
22800
23000
23200
23400
23600
23800
24000
24200
24400
24600
24800
25000


In [0]:
with open('/content/drive/My Drive/all data/1sentences_te' , 'wb') as f:
  pickle.dump(doc_vecs,f)

# LOAD FEATURES

In [2]:
import pickle
import numpy as np

with open('/content/drive/My Drive/Bert/Sen_feats','rb') as f:
  s_feats = pickle.load(f)
np.array(s_feats).shape

(50000, 768)

In [3]:
import pickle

with open('/content/drive/My Drive/Bert/Tok_feats','rb') as f:
  t_feats = pickle.load(f)
np.array(t_feats).shape

(50000, 768)

# NAIVE BAYES

In [0]:
from sklearn.naive_bayes import GaussianNB
import numpy as np

labels = np.zeros(25000)
labels[:12500] = 1

In [5]:
gnb_t = GaussianNB()
gnb_s = GaussianNB()

gnb_t.fit(t_feats[:25000],labels)

gnb_s.fit(s_feats[:25000],labels)

GaussianNB(priors=None, var_smoothing=1e-09)

In [7]:
y_pred_t = gnb_t.predict(t_feats[25000:])
y_pred_s = gnb_s.predict(s_feats[25000:])

np.mean(y_pred_s == labels)

0.83192

# SVM

In [9]:
import sklearn.svm as SVM
svm = SVM.LinearSVC(max_iter=500)
svm_s = svm.fit(s_feats[:25000],labels)
svm_t = svm.fit(t_feats[:25000],labels)


labels = np.zeros(25000)
labels[:12500] = 1

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
y_pred_s = svm_s.predict(s_feats[25000:])
y_pred_t = svm_t.predict(t_feats[25000:])

In [23]:
np.mean(y_pred_s == labels)

0.76872

# DECISION TREE

In [18]:
from sklearn.tree import DecisionTreeClassifier
dt_t = DecisionTreeClassifier()
dt_s = DecisionTreeClassifier()

dt_t.fit(t_feats[:25000],labels)
dt_s.fit(s_feats[:25000],labels)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [0]:
y_pred_t = dt_t.predict(t_feats[25000:])
y_pred_s = dt_s.predict(s_feats[25000:])

In [26]:
np.mean(y_pred_t == labels)

0.68924

# RANDOM FOREST

In [28]:
from sklearn.ensemble import RandomForestClassifier

rf_t = RandomForestClassifier()
rf_s = RandomForestClassifier()

rf_t.fit(t_feats[:25000],labels)
rf_s.fit(s_feats[:25000],labels)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_pred_t = rf_t.predict(t_feats[25000:])
y_pred_s = rf_s.predict(s_feats[25000:])

In [0]:
with open('/content/drive/My Drive/rf/y_pred_bertT','wb') as f:
  pickle.dump(y_pred_t,f)
with open('/content/drive/My Drive/rf/y_pred_bertS','wb') as f:
  pickle.dump(y_pred_s,f)
